In [4]:
## Program objective calculate dx dy dz to find dV 
## Program Written by Nicholas Munoz
## Take dV with respect to x / y / z which will provide dx dy dz then find the magnitude of dv = sqrt(dx**2+dy**2+dz**2)
## Sum up to N-1 (48/r12-24/r6)/r2)*x/r , y/r, z/r
import numpy as np
from scipy.optimize import minimize
import time
from scipy.spatial.distance import pdist, cdist
def LJ(r): 
    r6 = r**6
    r12 = r6*r6
    return 4*(1/r12 - 1/r6)

def total_energy(positions):
    """
    Calculate the total energy
    input:
    positions: 3*N array which represents the atomic positions
    output
    E: the total energy
    """
    E = 0
    N_atom = int(len(positions)/3)

    #positions = [x0, y0, z0, x1, y1, z1, .....  , xn, yn, zn]
    for i in range(N_atom-1):
        for j in range(i+1, N_atom):
            
            pos1 = positions[i*3:(i+1)*3]
            pos2 = positions[j*3:(j+1)*3]
            #print('pos1:  ', pos1)
            #print('pos2:  ', pos2)
            dist = np.linalg.norm(pos1-pos2)
            #print(i,j, dist)
            E += LJ(dist)
            
    return E
            
def init_pos(N, L=5):
    return L*np.random.random_sample((N*3,))

#def LJ_force(pos): Code used as a guideline
    #N_atom = int(len(pos)/3)
    #pos = np.reshape(pos,[N_atom, 3])
    #force = np.zeros([N_atom, 3])
    #for i, pos0 in enumerate(pos):
       # pos1 = pos.copy()
        #pos1 = np.delete(pos1, i, 0)
        #distance = cdist([pos0], pos1)
        #r = pos1 - pos0
        #r2 = np.power(distance, 2)
        #r6 = np.power(r2, 3)
        #r12 = np.power(r6, 2)
        #force[i] = np.dot((48/r12-24/r6)/r2, r)
        # force from the punish function mu*sum([x-mean(x)]^2)

    #return force.flatten()

def r_position():
    r_pos=[]
    for i in range(len(pos)//3):
        r_pos.append(np.linalg.norm(pos[i*3:(i+1)*3]))
        return r_pos
    
def Jacobian(pos): ## Jacobian function to pass into scipy 
    N_atom=int(len(pos)/3)
    force = np.zeros([N_atom, 3])
    for i in range(N_atom):
        pos0=pos[i*3:(i+1)*3]
        for j in range(N_atom):
            pos1=pos[j*3:(j+1)*3]
            if not any(pos0==pos1):
                distance_cart = pos0-pos1 ## cartesian coordinates x,y,z
                distance_r = np.linalg.norm(distance_cart)
                r14=distance_r**14
                r8= distance_r**8
                rdot = np.dot((-48/r14+24/r8),distance_cart)
                force[i]+=rdot
    return force.flatten()
    

In [11]:
## This section compares the energies found by the minimization to the actual values 
## which is fairly accurate when compared to the exact values


x_values=[]
N_atoms=[10,11,12,13,14,15,16,17,18,19,20] ## Number of atoms 
Energy =[-28.422532,-32.769570,-37.967600,-44.326801,-47.845157,-52.322627,-56.815742,-61.317995,-66.53049,-72.659782,-77.177043]
globstart=time.time() ## total time taken through the global scope of the program 

for n in N_atoms:
    function_val=[] 
    fval=0
    count=0
    start=time.time()
    
    while (fval-Energy[n-10]>0.7 and count<100): 
        pos=init_pos(n)
        
        
        res=minimize(total_energy,pos,jac=Jacobian,method='L-BFGS-B',tol=1e-4)
        fval=res.fun
        function_val.append(res.fun)
        
        count+=1
        
        print('\r Step:{:d} out of {:d}; value: {:.4} Time {:.2f} s'.format(count,200,res.fun,time.time()-start),flush=True,end='')
    
    
    if (fval-Energy[n-10]<=0.7): 
        ## This checks to see if our values passes an accepted bound
        ## by doing so it will pass out the value that is approximate to the minimized values
        
        print('\n The global min for N =',n,fval)
    
    else:
        
        print('\n The global min for N =',n,"is equal to",min(function_val))
        
        
print ("Time Taken", time.time()-globstart,"seconds")



 Step:47 out of 200; value: -28.42 Time 8.16 s
 The global min for N = 10 -28.421130366708297
 Step:100 out of 200; value: -25.15 Time 19.76 s
 The global min for N = 11 is equal to -31.794438283846215
 Step:100 out of 200; value: -36.09 Time 25.16 s
 The global min for N = 12 is equal to -36.30579603648524
 Step:74 out of 200; value: -44.33 Time 23.80 s
 The global min for N = 13 -44.32560738434508
 Step:98 out of 200; value: -47.83 Time 33.53 s
 The global min for N = 14 -47.83104631658523
 Step:31 out of 200; value: -52.32 Time 13.48 s
 The global min for N = 15 -52.319362151397094
 Step:23 out of 200; value: -56.81 Time 11.31 s
 The global min for N = 16 -56.81301928866909
 Step:100 out of 200; value: -54.97 Time 55.95 s
 The global min for N = 17 is equal to -60.43883745152172
 Step:31 out of 200; value: -66.28 Time 21.20 s
 The global min for N = 18 -66.27981896016105
 Step:100 out of 200; value: -63.33 Time 69.83 s
 The global min for N = 19 is equal to -70.1393596222474
 Step:1

In [12]:
## This section compares the energies found by the minimization to the actual values 
## which is fairly accurate when compared to the exact values


x_values=[]
N_atoms=[10,11,12,13,14,15,16,17,18,19,20] ## Number of atoms 
Energy =[-28.422532,-32.769570,-37.967600,-44.326801,-47.845157,-52.322627,-56.815742,-61.317995,-66.53049,-72.659782,-77.177043]
globstart=time.time() ## total time taken through the global scope of the program 

for n in N_atoms:
    function_val=[] 
    fval=0
    count=0
    start=time.time()
    
    while (fval-Energy[n-10]>0.8 and count<100): 
        pos=init_pos(n)
        
        
        res=minimize(total_energy,pos,jac=Jacobian,method='L-BFGS-B',tol=1e-4)
        fval=res.fun
        function_val.append(res.fun)
        
        count+=1
        
        print('\r Step:{:d} out of {:d}; value: {:.4} Time {:.2f} s'.format(count,200,res.fun,time.time()-start),flush=True,end='')
    
    
    if (fval-Energy[n-10]<=0.8): 
        ## This checks to see if our values passes an accepted bound
        ## by doing so it will pass out the value that is approximate to the minimized values
        
        print('\n The global min for N =',n,fval)
    
    else:
        
        print('\n The global min for N =',n,"is equal to",min(function_val))
        
        
print ("Time Taken", time.time()-globstart,"seconds")




 Step:10 out of 200; value: -28.42 Time 1.24 s
 The global min for N = 10 -28.42104753667399
 Step:100 out of 200; value: -10.37 Time 39.22 s
 The global min for N = 11 is equal to -31.79305186815207
 Step:16 out of 200; value: -37.97 Time 3.75 s
 The global min for N = 12 -37.965878398619125
 Step:56 out of 200; value: -44.33 Time 16.92 s
 The global min for N = 13 -44.32542277881107
 Step:31 out of 200; value: -47.84 Time 10.60 s
 The global min for N = 14 -47.83940834162192
 Step:69 out of 200; value: -52.31 Time 28.24 s
 The global min for N = 15 -52.31494035588718
 Step:100 out of 200; value: -52.0 Time 47.64 s
 The global min for N = 16 is equal to -55.902140040232986
 Step:3 out of 200; value: -61.32 Time 1.44 s
 The global min for N = 17 -61.31502266041476
 Step:100 out of 200; value: -60.7 Time 63.32 s
 The global min for N = 18 is equal to -65.68512485359769
 Step:100 out of 200; value: -61.48 Time 70.53 s
 The global min for N = 19 is equal to -70.70060807051566
 Step:9 out 

In [13]:
## This section compares the energies found by the minimization to the actual values 
## which is fairly accurate when compared to the exact values


x_values=[]
N_atoms=[10,11,12,13,14,15,16,17,18,19,20] ## Number of atoms 
Energy =[-28.422532,-32.769570,-37.967600,-44.326801,-47.845157,-52.322627,-56.815742,-61.317995,-66.53049,-72.659782,-77.177043]
globstart=time.time() ## total time taken through the global scope of the program 

for n in N_atoms:
    function_val=[] 
    fval=0
    count=0
    start=time.time()
    
    while (fval-Energy[n-10]>0.9 and count<100): 
        pos=init_pos(n)
        
        
        res=minimize(total_energy,pos,jac=Jacobian,method='L-BFGS-B',tol=1e-4)
        fval=res.fun
        function_val.append(res.fun)
        
        count+=1
        
        print('\r Step:{:d} out of {:d}; value: {:.4} Time {:.2f} s'.format(count,200,res.fun,time.time()-start),flush=True,end='')
    
    
    if (fval-Energy[n-10]<=0.9): 
        ## This checks to see if our values passes an accepted bound
        ## by doing so it will pass out the value that is approximate to the minimized values
        
        print('\n The global min for N =',n,fval)
    
    else:
        
        print('\n The global min for N =',n,"is equal to",min(function_val))
        
        
print ("Time Taken", time.time()-globstart,"seconds")




 Step:65 out of 200; value: -28.42 Time 10.31 s
 The global min for N = 10 -28.42056073097648
 Step:22 out of 200; value: -32.77 Time 4.37 s
 The global min for N = 11 -32.76507156114627
 Step:51 out of 200; value: -37.96 Time 12.95 s
 The global min for N = 12 -37.96381454110223
 Step:20 out of 200; value: -44.33 Time 6.21 s
 The global min for N = 13 -44.32558596859952
 Step:31 out of 200; value: -47.84 Time 11.45 s
 The global min for N = 14 -47.83588841924442
 Step:81 out of 200; value: -52.32 Time 36.32 s
 The global min for N = 15 -52.32075010281005
 Step:11 out of 200; value: -56.81 Time 5.44 s
 The global min for N = 16 -56.810795454051984
 Step:57 out of 200; value: -60.44 Time 31.36 s
 The global min for N = 17 -60.436591101968276
 Step:21 out of 200; value: -65.78 Time 13.55 s
 The global min for N = 18 -65.78338694052043
 Step:100 out of 200; value: -65.13 Time 69.59 s
 The global min for N = 19 is equal to -70.81145614432961
 Step:24 out of 200; value: -77.17 Time 17.75 s


In [14]:
## This section compares the energies found by the minimization to the actual values 
## which is fairly accurate when compared to the exact values


x_values=[]
N_atoms=[10,11,12,13,14,15,16,17,18,19,20] ## Number of atoms 
Energy =[-28.422532,-32.769570,-37.967600,-44.326801,-47.845157,-52.322627,-56.815742,-61.317995,-66.53049,-72.659782,-77.177043]
globstart=time.time() ## total time taken through the global scope of the program 

for n in N_atoms:
    function_val=[] 
    fval=0
    count=0
    start=time.time()
    
    while (fval-Energy[n-10]>0.95 and count<100): 
        pos=init_pos(n)
        
        
        res=minimize(total_energy,pos,jac=Jacobian,method='L-BFGS-B',tol=1e-4)
        fval=res.fun
        function_val.append(res.fun)
        
        count+=1
        
        print('\r Step:{:d} out of {:d}; value: {:.4} Time {:.2f} s'.format(count,200,res.fun,time.time()-start),flush=True,end='')
    
    
    if (fval-Energy[n-10]<=0.95): 
        ## This checks to see if our values passes an accepted bound
        ## by doing so it will pass out the value that is approximate to the minimized values
        
        print('\n The global min for N =',n,fval)
    
    else:
        
        print('\n The global min for N =',n,"is equal to",min(function_val))
        
        
print ("Time Taken", time.time()-globstart,"seconds")




 Step:6 out of 200; value: -27.54 Time 0.87 s
 The global min for N = 10 -27.54396667646283
 Step:8 out of 200; value: -32.76 Time 2.06 s
 The global min for N = 11 -32.76470892644937
 Step:33 out of 200; value: -37.97 Time 8.78 s
 The global min for N = 12 -37.96702780811039
 Step:100 out of 200; value: -35.46 Time 33.37 s
 The global min for N = 13 is equal to -41.47004416556085
 Step:2 out of 200; value: -47.84 Time 0.99 s
 The global min for N = 14 -47.84320586422154
 Step:6 out of 200; value: -52.32 Time 2.59 s
 The global min for N = 15 -52.31875098526893
 Step:45 out of 200; value: -56.81 Time 23.47 s
 The global min for N = 16 -56.80974630191514
 Step:71 out of 200; value: -61.09 Time 40.21 s
 The global min for N = 17 -61.09208666609307
 Step:29 out of 200; value: -65.8 Time 17.58 ss
 The global min for N = 18 -65.7990135524817
 Step:100 out of 200; value: -66.04 Time 71.06 s
 The global min for N = 19 is equal to -71.06391122472837
 Step:26 out of 200; value: -76.29 Time 21.2

In [15]:
## This section compares the energies found by the minimization to the actual values 
## which is fairly accurate when compared to the exact values


x_values=[]
N_atoms=[10,11,12,13,14,15,16,17,18,19,20] ## Number of atoms 
Energy =[-28.422532,-32.769570,-37.967600,-44.326801,-47.845157,-52.322627,-56.815742,-61.317995,-66.53049,-72.659782,-77.177043]
globstart=time.time() ## total time taken through the global scope of the program 

for n in N_atoms:
    function_val=[] 
    fval=0
    count=0
    start=time.time()
    
    while (fval-Energy[n-10]>0.85 and count<100): 
        pos=init_pos(n)
        
        
        res=minimize(total_energy,pos,jac=Jacobian,method='CG',tol=1e-4)
        fval=res.fun
        function_val.append(res.fun)
        
        count+=1
        
        print('\r Step:{:d} out of {:d}; value: {:.4} Time {:.2f} s'.format(count,200,res.fun,time.time()-start),flush=True,end='')
    
    
    if (fval-Energy[n-10]<=0.85): 
        ## This checks to see if our values passes an accepted bound
        ## by doing so it will pass out the value that is approximate to the minimized values
        
        print('\n The global min for N =',n,fval)
    
    else:
        
        print('\n The global min for N =',n,"is equal to",min(function_val))
        
        
print ("Time Taken", time.time()-globstart,"seconds")





 Step:11 out of 200; value: -28.42 Time 4.85 s
 The global min for N = 10 -28.422531893403814
 Step:11 out of 200; value: -32.77 Time 5.12 s
 The global min for N = 11 -32.765970089696566
 Step:3 out of 200; value: -37.97 Time 2.79 s
 The global min for N = 12 -37.96759956230493
 Step:18 out of 200; value: -44.33 Time 12.87 s
 The global min for N = 13 -44.326801419303585
 Step:9 out of 200; value: -47.85 Time 8.62 s
 The global min for N = 14 -47.84515678243911
 Step:16 out of 200; value: -52.32 Time 32.25 s
 The global min for N = 15 -52.32262726157659
 Step:100 out of 200; value: -47.42 Time 153.23 s
 The global min for N = 16 is equal to -55.90670239961136
 Step:4 out of 200; value: -61.31 Time 6.06 s
 The global min for N = 17 -61.30714608134491
 Step:36 out of 200; value: -65.69 Time 148.43 s
 The global min for N = 18 -65.6917485385215
 Step:7 out of 200; value: -72.66 Time 18.21 s
 The global min for N = 19 -72.65978245425298
 Step:12 out of 200; value: -77.18 Time 56.43 s
 The

In [16]:
## This section compares the energies found by the minimization to the actual values 
## which is fairly accurate when compared to the exact values


x_values=[]
N_atoms=[10,11,12,13,14,15,16,17,18,19,20] ## Number of atoms 
Energy =[-28.422532,-32.769570,-37.967600,-44.326801,-47.845157,-52.322627,-56.815742,-61.317995,-66.53049,-72.659782,-77.177043]
globstart=time.time() ## total time taken through the global scope of the program 

for n in N_atoms:
    function_val=[] 
    fval=0
    count=0
    start=time.time()
    
    while (fval-Energy[n-10]>0.85 and count<100): 
        pos=init_pos(n)
        
        
        res=minimize(total_energy,pos,jac=Jacobian,method='BFGS',tol=1e-4)
        fval=res.fun
        function_val.append(res.fun)
        
        count+=1
        
        print('\r Step:{:d} out of {:d}; value: {:.4} Time {:.2f} s'.format(count,200,res.fun,time.time()-start),flush=True,end='')
    
    
    if (fval-Energy[n-10]<=0.85): 
        ## This checks to see if our values passes an accepted bound
        ## by doing so it will pass out the value that is approximate to the minimized values
        
        print('\n The global min for N =',n,fval)
    
    else:
        
        print('\n The global min for N =',n,"is equal to",min(function_val))
        
        
print ("Time Taken", time.time()-globstart,"seconds")






 Step:14 out of 200; value: -28.42 Time 2.73 s
 The global min for N = 10 -28.422531893283256
 Step:9 out of 200; value: -32.77 Time 2.45 s
 The global min for N = 11 -32.765970089897976
 Step:12 out of 200; value: -37.97 Time 4.45 s
 The global min for N = 12 -37.96759956223479
 Step:41 out of 200; value: -44.33 Time 19.37 s
 The global min for N = 13 -44.326801419381184
 Step:15 out of 200; value: -47.85 Time 8.62 s
 The global min for N = 14 -47.845156782529024
 Step:20 out of 200; value: -52.32 Time 11.58 s
 The global min for N = 15 -52.32262726176893
 Step:22 out of 200; value: -56.82 Time 16.90 s
 The global min for N = 16 -56.81574178030849
 Step:11 out of 200; value: -61.3 Time 13.31 ss
 The global min for N = 17 -61.29676796751131
 Step:8 out of 200; value: -66.53 Time 8.75 s
 The global min for N = 18 -66.53094946284709
 Step:21 out of 200; value: -69.37 Time 25.17 s

KeyboardInterrupt: 